In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp
import pythainlp.word_vector

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus, clean

from model.new_sdc import NewSDC
from model.sdc import SDC

### Load Data

In [12]:
file_name = 'แก้ปัญหาฝุ่น'

corpus = load_corpus('../data/' + file_name + '.txt')
print('Total documents', len(corpus))

Total documents 229


In [13]:
# f = open('../data/tokenized/newmm/tokenized_' + file_name + '.txt')
f = open('../data/tokenized/deepcut/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

### Preprocess Corpus

#### Remove Words

In [14]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.common.thai_stopwords()
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1401 words
filter frequent words: 658 words
filter letter words: 654 words
filter stop words: 428 words


In [15]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Doc2Vec

In [6]:
def token_to_string(tokenized_list):
    string = str(tokenized_list)
    string.replace(',', ' ')
    return clean(string)

doc2vec_corpus = []
for doc in tokenized_corpus:
    array = pythainlp.word_vector.sentence_vectorizer(token_to_string(doc))[0]
    if any(numpy.isnan(array)):
        print('Nan')
        array = numpy.zeros(300)
    doc2vec_corpus.append(array)
doc2vec_corpus = pandas.DataFrame(doc2vec_corpus, dtype=float)

/home/kanoonsantikul/Documents/senior-project/venv/lib/python3.6/site-packages/pythainlp/word_vector/__init__.py:98: RuntimeWarning: invalid value encountered in true_divide
  vec /= len(words)


Nan


#### Dimension Reduction

In [16]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### Clustering

In [17]:
def get_document_vectors(corpus, value):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if value == 'bow':
        vectors = bow_corpus
    elif value == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        vectors = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(vectors):
        for id, score in doc:
            array[i, id] = score

        if value == 'bow' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [18]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [19]:
def eval_cluster(document_vectors, result):
    num_cluster = numpy.unique(result['predicted_label'])[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(document_vectors.iloc[i]))
        corpus_centroid.append(numpy.array(document_vectors.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [20]:
min_samples = 7
eps = 0.75
expand_rate = 0.05
epoch = 15

document_vectors = get_document_vectors(idx_corpus, 'bow')
# document_vectors = doc2vec_corpus

In [ ]:
max_compactness = 0
for i in range(epoch):
    model = NewSDC()
    _tpredicted_labels, marks = model.predict(document_vectors, min_samples, eps, expand_rate)

#     model = SDC()
#     _tpredicted_labels, marks = model.predict(document_vectors, min_samples, eps, expand_rate)
    
#     marks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(document_vectors)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=14).fit(document_vectors)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, marks)
    compactness, _tcentroids = eval_cluster(document_vectors, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
num_cluster = len(label_count)
print(label_count)

#### Iterative New SDC

In [21]:
centroids = None 
prev_label_count = None
model = NewSDC()
while True:
    predicted_labels, marks = model.predict(document_vectors, min_samples, eps, expand_rate, seeds=centroids)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(document_vectors, result)
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    if numpy.array_equal(label_count, prev_label_count):
        break
    prev_label_count = label_count
    centroids = centroids[1:]
    
    print(compactness)
    print(label_count)
num_cluster = len(label_count)

58.99646535408963
[229]


In [22]:
eps = 0.01
marks = None

while eps <= 1:
    max_compactness = 0
    for i in range(epoch):
        model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(document_vectors)
        _tpredicted_labels = model.labels_ + 1
        
        _tresult = generate_result(_tpredicted_labels, marks)
        compactness, _tcentroids = eval_cluster(document_vectors, _tresult)

        if compactness > max_compactness:
            max_compactness = compactness
            predicted_labels = _tpredicted_labels
            result = _tresult
            centroids = _tcentroids
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    print('%.2f' % eps, label_count)
    eps += 0.01

0.01 [229]
0.02 [229]
0.03 [229]
0.04 [229]
0.05 [229]
0.06 [229]
0.07 [229]
0.08 [229]
0.09 [229]
0.10 [229]
0.11 [229]
0.12 [229]
0.13 [229]
0.14 [229]
0.15 [229]
0.16 [229]
0.17 [229]
0.18 [229]
0.19 [229]
0.20 [229]
0.21 [229]
0.22 [229]
0.23 [229]
0.24 [229]
0.25 [229]
0.26 [229]
0.27 [229]
0.28 [229]
0.29 [229]
0.30 [213   9   7]
0.31 [213   9   7]
0.32 [213   9   7]
0.33 [213   9   7]
0.34 [213   9   7]
0.35 [213   9   7]
0.36 [213   9   7]
0.37 [212  10   7]
0.38 [212  10   7]
0.39 [212  10   7]
0.40 [212  10   7]
0.41 [205   7  10   7]
0.42 [204   7  11   7]
0.43 [199  10  20]
0.44 [198  11  20]
0.45 [189   8  11  21]
0.46 [187   8  12  22]
0.47 [186   9  12  22]
0.48 [183   9  14  23]
0.49 [181   9  16  23]
0.50 [165  10  28  20   6]
0.51 [152  61   9   7]
0.52 [147  65   9   8]
0.53 [141  71   9   8]
0.54 [139  73   9   8]
0.55 [137  75   9   8]
0.56 [125  88   9   7]
0.57 [124 105]
0.58 [113 116]
0.59 [109 120]
0.60 [ 91 132   6]
0.61 [ 88 135   6]
0.62 [ 84 145]
0.63 [ 81 

#### Grouping

In [15]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len(corpus))
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8]
8 7 0.8755721909893287
8 6 0.8586066011239457
8 5 0.8507223950545542
8 4 0.875182053956368
8 3 0.8589657405412545
8 2 0.8675732251896662
8 1 0.8875363722048335
7 6 0.9384452834460689
7 5 0.9141454743659437
7 4 0.9521101960285854
7 3 0.8930926351335264
7 2 0.9101058935577682
7 1 0.9658037508375265
6 5 0.9025615013232784
6 4 0.9354670147584043
6 3 0.9285334363748009
6 2 0.9410823368053696
6 1 0.9634998744557152
5 4 0.9131194410648852
5 3 0.8498011498062978
5 2 0.9004323324387875
5 1 0.9329955899312412
4 3 0.9034467677178339
4 2 0.9273065499085396
4 1 0.9723202135430872
3 2 0.9125217520384445
3 1 0.930462408162529
2 1 0.9438673547511558
[0, 1, 1, 1, 1, 1, 1, 1, 1]


### Result

In [16]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [17]:
# result.to_csv('../data/results/iterative_new_sdc/' + file_name + '_2.csv', index=False)

# result = pandas.read_csv('../data/results/iterative_new_sdc/' + file_name + '.csv')

print(eval_cluster(document_vectors, result)[0])
print(numpy.unique(result['predicted_label'], return_counts=True)[1])

186.78241587508978
[ 5 74 24 15 11 39 16 13 32]


In [18]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1), value=0)

54 This shows everything about Thailand
183 รัฐบาลโจร
187 รถล้านคัน 555    กูจำได้
220 +1
227 https//www.posttoday.com/social/general/577217


In [19]:
w2 = Widget(result, 'comment')

ToggleButtons(index=4, options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=4)

*7 การตัดแต่งต้นไม้ของ กทม. สักแต่ว่าตัดตัดจนมันเหี้ยนกุด ไม่เหลือสถาพของต้นไม้ประเทศไทยมีรุกขกรณ์ ที่มีความรู้ด้านการตัดแต่งต้นไม้ให้ได้ดีและสวยงามแค่ปรับปรุง ปรับเปรี่ยนการตัดแต่งต้นไม้ เราก็จะได้ประสิทธิภาพการกรองฝุ่นและมลพิษ กลับมามากมายแล้ว
24 คือแบบ ไม่มีการตายให้เห็นปัจจุบันทันด่วน ก็เลยไม่วิตกว่าเรื่องนี้ มันเป็นวิกฤติของประเทศ...มังคะ..
*43 ทุกวันนี้ลูกก็ยังได้เข้าแถวหน้าเสาธง ห้องเรียนยังเปิดประตูหน้าต่าง ไปค่ายลูกเสือทำกิจกรรมกลางแจ้งเฉย ชีวิตคนไทยมันก็เป็นแบบนี้แหละ ต้องดูแลตัวเองจริงๆ ถ้าหวังพึ่งพารัฐก็เครียดไปเปล่าๆ แต่ก็ขอบคุณหมอนะที่เขียนบทความแสดงออกถึงความห่วงใย
*55 ขอบคุณ​คุณหมอค่ะ​ รัฐบาลไม่ประกาศ​สิ่งที่เป็นรูปธรรม​อะไรสักอย่าง​ มีแต่เราที่ต้องป้องกันตัว​เอง​ เฉยแบบนี้​ ก็ไม่มีการตื่นตัว​ว่ามันอันตรายแล้ว​ รู้ทั้ง​รู้​แต่กลับ​เฉย
78 คุณหมอคะ นอกจากรัฐควรเร่งปลูกต้นไม้. รัฐบาลควรออกกฏหมายรักษาต้นไม้ที่มีอยู่อย่างเคร่งครัดด้วยค่ะ. เห็นตัดกันเป็นว่าเล่น ต้นใหญ่ๆทั้งนั้น 😪
*82 มันคือฝุ่นที่มองไม่เห็น และมีผลระยะยาว เลยทำให้คนไม่ตื่นกลัว ส่วนรัฐก็บอกไม่มีอะไร สงสารเด็กท

In [20]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0)

54 []
183 ['รัฐบาล']
187 ['รถ', 'ล้าน', 'คัน']
220 []
227 []


In [19]:
seed = 0
compare = 0

a = numpy.array(document_vectors.iloc[seed]).reshape(1, -1)
b = numpy.array(document_vectors.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))

print(idx_corpus[seed])
print(corpus[seed])
print(idx_corpus[compare])
print(corpus[compare])

[[1.]]
['ประกาศ', 'หยุด', 'เรียน', 'ระดับ', 'ชั้น', 'บริหาร', 'โรง', 'เรียน', 'ประกาศ', 'ชม']
รร.รุ่งอรุณประกาศหยุดเรียนทุกระดับชั้น ชื่นชมผู้บริหารโรงเรียนในการตัดสิ้นใจค่ะ ประกาศเมื่อราว 1 ชม.ที่ผ่านมาค่ะ
['ประกาศ', 'หยุด', 'เรียน', 'ระดับ', 'ชั้น', 'บริหาร', 'โรง', 'เรียน', 'ประกาศ', 'ชม']
รร.รุ่งอรุณประกาศหยุดเรียนทุกระดับชั้น ชื่นชมผู้บริหารโรงเรียนในการตัดสิ้นใจค่ะ ประกาศเมื่อราว 1 ชม.ที่ผ่านมาค่ะ
